In [1]:
import fastai
from fastai.vision.all import (
    DataLoaders, Learner, URLs, cnn_learner, resnet18, 
    accuracy, ImageDataLoaders, untar_data
)
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import functional as F

from mnist.mean_classifier import MeanClassifier, mse, mae
from mnist import learner
from mnist import data

In [2]:
train_dl, test_dl = data.load_mnist()
dataloaders = DataLoaders(train_dl, test_dl)

regression_dl = data.load_linear()

In [3]:
for loss_func in mse, mae:
    clf = MeanClassifier(loss=loss_func)
    clf.fit(train_dl)
    print(f'Mean Classifier with {loss_func.__name__.upper()} loss had an accuracy of {clf.score(test_dl):.4}')

Mean Classifier with MSE loss had an accuracy of 0.8187
Mean Classifier with MAE loss had an accuracy of 0.6704


# Gradient Descent

In [4]:
from mnist import optimiser


def r_2(y_pred, y_true):
    u = ((y_true - y_pred) ** 2).sum()
    v = ((y_true - y_true.mean()) ** 2).sum()
    return (1 - (u/v)).item()


for loss_func in F.mse_loss, F.l1_loss:
    m = torch.nn.Linear(1, 1)
    reg = learner.Learner(lr=1e-3, epochs=100, loss=loss_func, optimiser=optimiser.SGD, model=m)
    reg.fit(regression_dl)
    batch_scores = [r_2(reg.predict(X), y) for X, y in regression_dl]
    score = np.mean(batch_scores)
    print(f'Gradient Descent Regressor with {loss_func.__name__.upper()} loss had an R^2 of {score:.4}')

Gradient Descent Regressor with MSE_LOSS loss had an R^2 of 0.9785
Gradient Descent Regressor with L1_LOSS loss had an R^2 of 0.9784


# Classifier

In [5]:
model = torch.nn.Linear(28*28, 10)
clf = learner.Learner(lr=1, epochs=10, optimiser=optimiser.SGD, model=model, loss=nn.CrossEntropyLoss())
clf.fit(train_dl)

def accuracy(preds, y):
    return (preds.argmax(-1) == y).float().mean()

score = np.mean([accuracy(clf.predict(X), y) for X, y in test_dl])
print(f'Gradient Descent Classifier with custom SGD had an accuracy of {score:.4}')

Gradient Descent Classifier with custom SGD had an accuracy of 0.9193


In [6]:
# trying with pytorch's SGD:
clf = learner.Learner(epochs=10, lr=1, optimiser=torch.optim.SGD, loss=nn.CrossEntropyLoss(), model=torch.nn.Linear(28*28, 10))
clf.fit(train_dl)
score = np.mean([accuracy(clf.predict(X), y) for X, y in test_dl])
print(f"Gradient Descent Classifier with torch's SGD had an accuracy of {score:.4}")

Gradient Descent Classifier with torch's SGD had an accuracy of 0.9121


In [7]:
learn = Learner(dataloaders, torch.nn.Linear(28*28,10), opt_func=fastai.optimizer.SGD, loss_func=nn.CrossEntropyLoss(), metrics=accuracy)
learn.fit(10, lr=1)

epoch,train_loss,valid_loss,accuracy,time
0,0.349351,0.320776,0.906500,00:06
1,0.312170,0.293248,0.916700,00:06
2,0.287037,0.314138,0.910000,00:07
3,0.279780,0.294972,0.917700,00:06
4,0.278093,0.286686,0.918500,00:06
5,0.280224,0.297022,0.919200,00:06
6,0.279300,0.285074,0.920400,00:06
7,0.274514,0.278062,0.920000,00:06
8,0.280260,0.295389,0.918200,00:06
9,0.269228,0.277691,0.921700,00:06


# Moving to Neural Networks

In [8]:
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 10),
)

In [9]:
from mnist import optimiser
clf = learner.Learner(epochs=10, lr=1, optimiser=optimiser.SGD, loss=nn.CrossEntropyLoss(), model=simple_net)
clf.fit(train_dl)
score = np.mean([accuracy(clf.predict(X), y) for X, y in test_dl])
print(f"Neural Network had an accuracy of {score:.4}")

Neural Network had an accuracy of 0.9509
